# Data Processing
## Index
1. Figures from multiple data sets
2. Fitting Critical exponents (m, chi, c_v)
3. Fitting Dynamical Critical Exponent
4. Simulation performance


## 1. Figures from multiple data sets

In [6]:
# Figures for c_v, chi, and m^2 are made from three data sets, L = 10, L = 20, L = 40

import numpy as np
import matplotlib.pyplot as plt

# Specify where to find data sets
data_dir = './exported_data/'

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'triple_2000_' 
fig_name = fig_dir + identifier

# Properties of the data
data_names = ['c_v', 'magnetisation', 'chi']
L_range = [10, 20, 40]
T_data = 81

# Initialization
y_data = np.zeros((len(L_range), T_data))
x_data = np.zeros((len(L_range), T_data))

# Plot markup
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

for data_name in data_names:
    
    # Import the data from the selected data sets 
    for i, L in enumerate(L_range):
        name = 'saved_data_L' + str(L) +'_SW_2000.npz'
        npzfile = np.load(data_dir + name)
        y_data[i] = npzfile[data_name][:,0]
        y_err[i] = npzfile[data_name][:,1]
        x_data[i] = npzfile['temperature'][:,0]
    
    # Plot imported data
    markers = ['+', 'x', '.']
    for k, x in enumerate(x_data):
        # Normal plot or errorbar plot
        plt.plot(x, y_data[k], markers[k], markersize = 6)
        #plt.errorbar(x, y_data[k], yerr = y_err[k], fmt = markers[k], markersize = 6, capsize = 4)
        plt.xlabel('$\mathrm{k_B T/J}$', fontsize=18)
    plt.legend(['L = 10', 'L = 20', 'L = 40'])

    # Plot markup
    if data_name == 'c_v':
        plt.ylabel('$\mathrm{C_v}$', fontsize=18)
    if data_name == 'magnetisation':
        plt.ylabel('$ m^{2} $', fontsize=18)   
    if data_name == 'chi':
        plt.ylabel('$\chi$', fontsize=18)
    
    plt.tight_layout()
    full_dir_name = fig_name + data_name + '.png'
    plt.savefig(fig_name + data_name + '.png')
    plt.close()
print('Figures are saved to:', fig_dir)

Figures are saved to: ./exported_figs/


## 2. Fitting Critical exponents (m, chi, c_v)

### 2.1 Needed sub functions:

In [ ]:
def crit_temp(data_dir):
    '''Gives the transition/critical temperature from the measured specific
    heat in terms of k_B*T/J.
    
    Parameters
    ----------
    data_dir : datafile ~.npz
        contains the 'temperature' 'magnetic_field' 'c_v' 'chi' arrays
    
    Returns
    -------
    T_c: float
        the critical temperature    
    '''
    
    data = np.load(data_dir)
    T = data['temperature'].reshape(np.shape(data['temperature'])[0])
    c_v = data['c_v'][:,0]
    T_c = T[c_v==max(c_v)]
    
    return T_c

In [ ]:
def load_data(data_dir, quantity):
    '''Loads all data from data_dir and assigns data of specific quantity to variables
    
    Parameters
    ----------
    data_dir : datafile ~.npz
        contains the 'temperature' 'magnetic_field' 'c_v' 'chi' 'magnetisation' arrays
    quantity : str
        quantity of interest
    
    Returns
    -------
    xdata : 1D array
        temperatures from data_dir
    ydata : 1D array
        quantity from data_dir
    y_err : 1D array
        standard deviation of ydata entries        
    '''
    
    data = np.load(data_dir)
    xdata = data['temperature'].reshape(np.shape(data['temperature'])[0])
    ydata = data[quantity][:,0]
    y_err = data[quantity][:,1]
    
    return xdata, ydata, y_err

In [ ]:
def plot_function(data_dir, quantity, LOG):
    '''Gives plot of the function in loglog or normal scale. X-axis is
    translated with critical temperature T_c when the loglog plot is selected.
    
    Parameters
    ----------
    data_dir : datafile ~.npz
        contains the 'temperature' 'magnetic_field' 'c_v' 'chi' arrays
    quantity : Str
        string which specifies which physical quantity must be fitted
    LOG : boolean
        Turns loglog plot on or off
        
    Returns
    -------
    plot of data provided
    '''
    
    xdata, ydata, y_err = load_data(data_dir, quantity)
    
    T_c = crit_temp(data_dir)
    
    #if quantity == 'magnetisation':
    
    if LOG:
        plt.loglog(xdata[(xdata-T_c)>0]-T_c, ydata[(xdata-T_c)>0], 'bx')
        plt.loglog(abs(xdata[(xdata-T_c)<0]-T_c), ydata[(xdata-T_c)<0], 'rx')
        plt.grid()
    else:
        plt.plot(xdata[(xdata-T_c)>0]-T_c, ydata[(xdata-T_c)>0], 'bx')
        plt.plot(xdata[(xdata-T_c)<0]-T_c, ydata[(xdata-T_c)<0], 'rx')
        
    plt.tight_layout()
    plt.show()
        

### 2.2 Actual fit function

In [ ]:
def fit_function(data_dir, quantity, fit_range, plotYN):
    '''Gives the best fit to quantity with non-linear least squares
    
    Parameters
    ----------
    data_dir : datafile ~.npz
        contains the 'temperature' 'magnetic_field' 'c_v' 'chi' arrays
    quantity : Str
        string which specifies which physical quantity must be fitted
    fit_range: sequence
        [LB, UB], the lower and upper boundary of the interval to be
        fitted w.r.t T_c
    plotYN : boolean
        turns plotting of fit and original function on or off
              
    Returns
    -------
    popt: 1D array
        Optimal values for parameters so sum of the squared residuals is minimized
    fit_err: sequence
        Standard deviation of the optimal values for the parameters
    xdata_fit_plot : 1D array
        x-axis for plotting the fitted function
    ydata_fit_plot : 1D array
        results for the fit by using xdata_fit_plot
    '''
    
    # Initialisation of plot design and parameters
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.rc('font', size=18)
    plt.xlabel('$\mathrm{k_B (T-T_c)/J}$')
    
       
    # Loading data and general functions (and generate transition temperature (T_c))
    from scipy.optimize import curve_fit
    
    data = np.load(data_dir)
    xdata = data['temperature'].reshape(np.shape(data['temperature'])[0])
    ydata = data[quantity][:,0]
    y_err = data[quantity][:,1]
    
    T_c = crit_temp(data_dir)
    
    # Fitting
    ## Selecting data for fit
    indices = np.where(((xdata-T_c)>fit_range[0]) & ((xdata-T_c)<fit_range[1]))
    xdata_fit = xdata[indices] - T_c
    ydata_fit = ydata[indices]
    y_err_fit = y_err[indices]

    ## Select fitting model
    if quantity == 'magnetisation':
        from data_processing import f_magnetisation as f
        plt.ylabel('m', fontsize=18)
    if quantity == 'c_v':
        from data_processing import f_cv as f
        plt.ylabel('$\mathrm{C_v}$', fontsize=18)
    if quantity == 'chi':
        from data_processing import f_chi as f
        plt.ylabel('$\mathrm{\chi}$', fontsize=18)
    
    ## Actual fitting
    popt, pcov = curve_fit(f, abs(xdata_fit), ydata_fit, sigma=y_err_fit)
    fit_err = np.sqrt(np.diag(pcov))

    # Generate fit plot data
    xdata_fit_plot = np.linspace(xdata_fit[0], xdata_fit[-1], 1000)
    ydata_fit_plot = f(abs(xdata_fit_plot), *popt)
        
    # Plotting original function and fit
    if plotYN:
                
        # Actual plotting
        plt.loglog(xdata[(xdata-T_c)>0]-T_c, ydata[(xdata-T_c)>0], 'bx', markersize=7)
        plt.loglog(abs(xdata[(xdata-T_c)<0]-T_c), ydata[(xdata-T_c)<0], 'rx', markersize=7)
        plt.loglog(abs(xdata_fit_plot),ydata_fit_plot, 'k-', alpha = 0.9, label = 'Fit +')
        plt.grid()
   
    return popt, fit_err, xdata_fit_plot, ydata_fit_plot

### 2.3 Determine best fit (incl. plot)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook

boundaries_neg = np.array([-0.46, -0.074])
popt_neg, fit_err_neg, xdata_fit_neg, ydata_fit_neg = fit_function('./exported_data/saved_data_L40_SW_5000.npz', 'chi', boundaries_neg, False)

boundaries_pos = np.array([0.124, 0.576])
popt_pos, fit_err_pos, xdata_fit_plot_pos, ydata_fit_plot_pos = fit_function('./exported_data/saved_data_L40_SW_5000.npz', 'chi', boundaries_pos, True)

plt.loglog(abs(xdata_fit_neg),ydata_fit_neg, 'k--', alpha = 0.9 ,label = 'Fit -')
plt.legend()
plt.tight_layout()
plt.show()

#plt.savefig('./exported_figs/L40_SW_5000' + '_chi_fitted.png')

## 3. Fitting Dynamical Critical Exponent

In [1]:
# Imports chi data, fits to obtain z, plots fit including 3 data sets

import numpy as np
import matplotlib.pyplot as plt
from data_processing import f_z, fit_funct_z

# Specify where to find data sets 
data_dir = './exported_data/'

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'triple_fit_'
fig_name = fig_dir + identifier

# Data specifications
data_names = ['chi'] # Quantity
L_range = [10, 15, 20, 25, 30, 35, 40] # Data sets
T_data = 81 # Temperature steps 

# Initialization 
y_data = np.zeros((len(L_range), T_data))
y_err = np.zeros((len(L_range), T_data))
x_data = np.zeros((len(L_range), T_data))
x_err = np.zeros((len(L_range), T_data))
top_xy = np.zeros((len(L_range), 2))
top_xy_err = np.zeros((len(L_range), 2))

# Plot markup 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

for data_name in data_names:
    for i, L_i in enumerate(L_range):
        # Import data sets 
        
        name = 'saved_data_L' + str(L_i) +'_SW_2000.npz'
        npzfile = np.load(data_dir + name)
        y_data[i] = npzfile[data_name][:,0]
        y_err[i] = npzfile[data_name][:,1]
        
        j = np.argmax(y_data[i])    # Identify maximum of the dataset
        top_xy [i, 1]= y_data[i, j] # Sore values of the maximum 
        top_xy_err[i,1] = y_err[i,j]
        
        x_data[i] = npzfile['temperature'][:,0]
        top_xy[i, 0] = x_data[i,j]
        
        top_xy_err[i,0] = abs(npzfile['temperature'][j + 1,0] - npzfile['temperature'][j,0])/2
        
ii = 0
markers = ['+', 'x', '.']
for k, L_i in enumerate(L_range):
    if L_i in [10, 20, 40]:
        # Plot data for 3 data sets
        plt.errorbar(x_data[k], y_data[k], yerr=y_err[k], fmt=markers[ii%3], label = 'L = %.f'%L_i , markersize = 6, capsize = 4)
        ii += 1
plt.xlabel('$\mathrm{k_B T/J}$', fontsize=18)
plt.ylabel('$\chi$', fontsize=18)

# Fit to the data 
popt, fit_err = fit_funct_z(f_z, L_range, top_xy[:,0], top_xy_err[:,0], (-np.inf, np.inf))
popt2, fit_err2 = fit_funct_z(f_z, L_range, top_xy[:,1], top_xy_err[:,1], (-np.inf, np.inf))

# Calculation using the fitting results 
L_size = np.linspace(10, 40, 100)
x = f_z(L_size, *popt)
x_max = f_z(L_size, *popt + fit_err)
x_min = f_z(L_size, *popt - fit_err) 

y = f_z(L_size, *popt2)
y_max = f_z(L_size, *popt2 + fit_err2)
y_min = f_z(L_size, *popt2 - fit_err2)

# Plotting 
plt.plot(x, y, 'b-', label = 'Fit')
plt.legend()
plt.plot(x_max, y_max, 'b--')
plt.plot(x_min, y_min, 'b--')
plt.tight_layout()
print('z =', np.round(-popt[0], 2), '(', int(np.round(100*fit_err[0],0)), ')' )
plt.savefig(fig_name + data_name + '.png')
print('Figures are saved to:', fig_dir)
plt.close()

z = 0.19 ( 4 )
Figures are saved to: ./exported_figs/


## 4. Simulation performance

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from data_processing import f_sim, fit_funct_sim

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'sim_performance'
fig_name = fig_dir + identifier

# Simulation data from general runs, all for the same ammount of MC steps.
time_sf = np.array([5.00, 20.66, 43.34 ,81.8])
L_sf = np.array([10, 20, 30, 40])
time_sw = np.array([3.1, 11.3, 25.8, 44.23])
L_sw = L_sf

norm = max(time_sf)
time_sf = time_sf/norm
time_sw = time_sw/norm

# Plot markup 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

plt.plot(L_sf, time_sf, 'b+', markersize = 6, label = 'Metropolis')
plt.plot(L_sw, time_sw, 'x', color = 'orange', markersize = 6, label = 'Swendsen-Wang')
plt.legend()
# Fitting 
popt, fit_err = fit_funct_sim(f_sim, L_sf, time_sf)
popt2, fit_err2 = fit_funct_sim(f_sim, L_sw, time_sw)

L_range = np.linspace(10, 40, 100)
plt.plot(L_range, f_sim(L_range, *popt), 'b--', label = 'Metropolis fit')
plt.plot(L_range, f_sim(L_range, *popt2), '-.', color = 'orange', label = 'Swendsen-Wang fit')
plt.xlabel('L')
plt.ylabel('t')
plt.tight_layout()

print('Rough Metropolis behaviour:', np.round(popt[1], 2))
print('Rough Swendsen-Wang behaviour:', np.round(popt2[1], 2))

plt.savefig(fig_name + '.png')
print('Figure are saved to:', fig_dir)
plt.close()

Rough Metropolis behaviour: 2.17
Rough Swendsen-Wang behaviour: 1.91
Figure are saved to: ./exported_figs/
